In [1]:
import backtrader as bt
import pandas as pd
import numpy as np
# from get_score import get_score_ch
from sqlalchemy import create_engine
import talib as ta
import datetime
import pandas as pd
engine= create_engine('sqlite:///../../../dataset/us/us_ticker_price_yf.db')

In [7]:
ticker = 'AAPL'
# ticker = 'INTC'
date = '2020-06-11'
# date = '2020-01-09'
df = pd.read_sql('{}'.format(ticker), engine)
df.rename(columns={'hgih':'high'},inplace=True)
df['chg_pct'] = ((df.close - df.open)/df.open)*100
df['close'] = df['close'].apply(lambda x: round(x,3))
df['VMA20'] = ta.SMA(df.volume, timeperiod=20)
df['VMA50'] = ta.SMA(df.volume, timeperiod=50)

In [8]:
df = df.tail(600)
df.index = range(len(df))
df_length = len(df)
df_length

600

In [9]:
temp = df[50:100]
# n个bar内，升高的天数和降低的天数
increase_day_count  = temp[temp.chg_pct > 0].datetime.count()
decrease_day_count  = temp[temp.chg_pct < 0].datetime.count()
increase_day_count, decrease_day_count

(29, 21)

In [10]:
# n个bar内，升高天数得volume均值和降低天数得volume均值
increase_day_volume = round(temp[temp.chg_pct > 0].volume.mean(),3)
decrease_day_volume = round(temp[temp.chg_pct < 0].volume.mean(),3)
increase_day_volume, decrease_day_volume

(129124524.138, 148713580.952)

In [15]:
increase_date_count = temp.query('volume > VMA50 and chg_pct > 0').datetime.count()
decrease_date_count = temp.query('volume > VMA50 and chg_pct < 0').datetime.count()
increase_date_count, decrease_date_count

(5, 7)

In [58]:
# 关闭警告的设置方法
pd.set_option('mode.chained_assignment', None)
inputdate = '2020-06-11'

targetindex = df[df.datetime == '{}'.format(inputdate)].index.values[0]
startindex = targetindex - 50
temp = df[startindex: targetindex]
# 
increase_volume = round(temp[temp.chg_pct > 0].volume.mean(),3)
decrease_volume = round(temp[temp.chg_pct < 0].volume.mean(),3)
vol_diff = round((increase_volume - decrease_volume) / 1000,0)
# 
increase_rate = round(temp.query('chg_pct > 0 and volume > VMA20').datetime.count() / temp.query('chg_pct > 0').datetime.count(),3)
decrease_rate = round(temp.query('chg_pct < 0 and volume > VMA20').datetime.count() / temp.query('chg_pct < 0').datetime.count(),5)
rate_diff = round((increase_rate - decrease_rate)*1000,3)

print('vol diff: {}, rate diff: {}'.format(vol_diff, rate_diff))

vol diff: -12033.0, rate diff: -3.22


In [63]:
temp['factor'] = round(temp.volume / temp.VMA20, 3)
increase_rate_fac = temp.query('chg_pct > 0 and volume > VMA20').factor.sum() / temp.query('chg_pct > 0').factor.sum()
decrease_rate_fac = temp.query('chg_pct < 0 and volume > VMA20').factor.sum() / temp.query('chg_pct < 0').factor.sum()

0.31169899252964156